In [ ]:
import logging
import os

from hirundo import (
    GitRepo,
    HirundoCSV,
    LabelingType,
    OptimizationDataset,
    StorageGit,
    StorageIntegration,
    StorageTypes,
)
from hirundo.git import GitPlainAuthBase

logger = logging.getLogger(__name__)

unique_id = os.getenv("UNIQUE_ID", "").replace(".", "-").replace("/", "-")

run_ids = [
    run.run_id
    for run in OptimizationDataset.list_runs()
    if run.run_id and run.name.startswith("TEST-STT-MASC-dataset")
]
for run_id in run_ids:
    OptimizationDataset.cancel_by_id(run_id)
dataset_ids = [
    dataset.id
    for dataset in OptimizationDataset.list_datasets()
    if dataset.name.startswith("TEST-STT-MASC-dataset")
]
for dataset_id in dataset_ids:
    OptimizationDataset.delete_by_id(dataset_id)
storage_integration_ids = [
    storage_integration.id
    for storage_integration in StorageIntegration.list()
    if storage_integration.name.startswith("STT-MASC-dataset")
]
for storage_integration_id in storage_integration_ids:
    StorageIntegration.delete_by_id(storage_integration_id)
git_repo_ids = [
    git_repo.id
    for git_repo in GitRepo.list()
    if git_repo.name.startswith("STT-MASC-dataset")
]
for git_repo_id in git_repo_ids:
    GitRepo.delete_by_id(git_repo_id)

test_storage_git = StorageGit(
    repo=GitRepo(
        name=f"STT-MASC-dataset{unique_id}",
        repository_url="https://huggingface.co/datasets/hirundo-io/MASC",
        plain_auth=GitPlainAuthBase(
            username="blewis-hir",
            password=os.environ["HUGGINGFACE_ACCESS_TOKEN"],
        ),
    ),
    branch="main",
)
test_dataset = OptimizationDataset(
    name=f"TEST-STT-MASC-dataset{unique_id}",
    labeling_type=LabelingType.SPEECH_TO_TEXT,
    language="ar",
    storage_integration=StorageIntegration(
        name=f"STT-MASC-dataset{unique_id}",
        type=StorageTypes.GIT,
        git=test_storage_git,
    ),
    data_root_url=test_storage_git.get_url("/wavs"),
    labeling_info=HirundoCSV(
        csv_url=test_storage_git.get_url("/meta.csv"),
    ),
)

# test_dataset.run_optimization()

In [2]:
results = test_dataset.check_run()
results.suspects.to_csv("ar-on-prem-audio-test.csv")

Optimization run queued and not yet started:   0%|          | 0.0/100.0 [00:00<?, ?it/s]

RemoteProtocolError: peer closed connection without sending complete message body (incomplete chunked read)